In [1]:
from pygod.utils import load_data
from typing import List
from gad_adversarial_robustness.gad.dominant.dominant import Dominant
from gad_adversarial_robustness.gad.dominant.dominant_lip import DominantLIP
from gad_adversarial_robustness.utils.graph_utils import load_anomaly_detection_dataset, get_n_anomaly_indexes
import argparse
import torch
from torch_geometric.utils import to_dense_adj
import numpy as np
from gad_adversarial_robustness.poison.greedy import multiple_AS, poison_attack
import copy

In [2]:
data = load_data("inj_cora")
truth: List[int] = data.y.bool()

adj, attrs, label, adj_label = load_anomaly_detection_dataset(data)
adj = torch.FloatTensor(adj)
adj_label = torch.FloatTensor(adj_label)
attrs = torch.FloatTensor(attrs)

class Args:
    def __init__(self, hidden_dim=64, epoch=100, lr=5e-3, dropout=0.3, alpha=0.8, device='cpu'):
        self.hidden_dim = hidden_dim
        self.epoch = epoch
        self.lr = lr
        self.dropout = dropout
        self.alpha = alpha
        self.device = device

# Create an instance of the Args class with default values
args = Args()


dominant = Dominant(feat_size = attrs.size(1), hidden_size = args.hidden_dim, dropout = args.dropout, device = args.device, adj=adj, adj_label=adj_label, attrs=attrs, label=label)
dominant.fit(args)


Epoch: 0000 train_loss= 2.13339 train/struct_loss= 8.67259 train/feat_loss= 0.49859
Score size: (2708,)
Epoch: 0000 Auc 0.8475976992048722
Epoch: 0001 train_loss= 1.46249 train/struct_loss= 6.06963 train/feat_loss= 0.31070
Epoch: 0002 train_loss= 1.06913 train/struct_loss= 4.31199 train/feat_loss= 0.25842
Epoch: 0003 train_loss= 0.83430 train/struct_loss= 3.14609 train/feat_loss= 0.25635
Epoch: 0004 train_loss= 0.70848 train/struct_loss= 2.51666 train/feat_loss= 0.25643
Epoch: 0005 train_loss= 0.65028 train/struct_loss= 2.22744 train/feat_loss= 0.25599
Epoch: 0006 train_loss= 0.63538 train/struct_loss= 2.15234 train/feat_loss= 0.25614
Epoch: 0007 train_loss= 0.63284 train/struct_loss= 2.13910 train/feat_loss= 0.25627
Epoch: 0008 train_loss= 0.63236 train/struct_loss= 2.13782 train/feat_loss= 0.25599
Epoch: 0009 train_loss= 0.63249 train/struct_loss= 2.13867 train/feat_loss= 0.25595
Epoch: 0010 train_loss= 0.63253 train/struct_loss= 2.13831 train/feat_loss= 0.25608
Score size: (2708,)
E

In [8]:
import numpy as np
from gad_adversarial_robustness.utils.graph_utils import prepare_graph
dense_adj = dominant.adj
amount_of_nodes = data.num_nodes

_, adj, _ = prepare_graph(data)
dense_adj = adj.to_dense()  #Fill in zeroes where there are no edges

print("Create poison compatible adjacency matrix...")
triple = []
for i in range(amount_of_nodes):
    for j in range(i + 1, amount_of_nodes):
        triple.append([i, j, dense_adj[i,j]])  #Fill with 0, then insert actual after

#print(type(triple))
#triple = np.array([t.detach().numpy() for t in triple])
#np.array(triple)




# These are the nodes we try reduce the active subnetwork score for (disguising anonomaly nodes)
target_node_lst = get_n_anomaly_indexes(truth, 10)
print(type(target_node_lst)), print(f'target node list: {target_node_lst}'), print(target_node_lst)

print("Making model...")
model = multiple_AS(target_lst = target_node_lst, n_node = amount_of_nodes, device = 'cpu')
budget = 2  # The amount of edges to change



Create poison compatible adjacency matrix...
<class 'list'>


AttributeError: 'list' object has no attribute 'detach'

In [5]:

# shape of triple
print(f'shape of triple: {len(triple)}')
print(type(triple))
print(type(triple[2][1]))
print("Starting attack...")
#triple = triple.detach().cpu().numpy()

#triple = torch.tensor(triple, requires_grad=True)


adj_adversary, _, _ = poison_attack(model, triple, budget)

print("Converting to compatible tensor...")

# Create Edge Index'
edge_index = torch.tensor([[],[]])

# Transpose it to make shape compatible
transposed_adj_adversary = torch.transpose(adj_adversary, 0, 1)

for i in range(len(adj_adversary)):
    if(adj_adversary[i][2] != 0):   #If edge value is not 0 (no edge)
        #Add edge to edge index, choosing first 2 elements (edges), and then the ith edge
        edge_index = torch.cat((edge_index, transposed_adj_adversary[:2, i:i+1]), -1)
        # Dataset uses edges both ways so add reverse edge as well
        edge_index = torch.cat((edge_index, torch.flip(transposed_adj_adversary[:2, i:i+1], dims=[0])), -1)


# Make new data object with new edge index
data_after_poison = copy.deepcopy(data)
edge_index = edge_index.type(torch.int64)
data_after_poison.edge_index = edge_index






shape of triple: 3665278
<class 'list'>
<class 'int'>
Starting attack...
triple copy type: <class 'list'>


TypeError: expected np.ndarray (got list)